In [2]:
import numpy as np 
import pandas as pd 
import math 
from sklearn import preprocessing
import gmplot
import matplotlib.pyplot as plt
from time import time
import operator
from datetime import datetime

In [3]:
from geopy.geocoders import Nominatim
import overpass
import folium
from folium import plugins
from folium.plugins import HeatMap
geolocator = Nominatim(user_agent="TFG")
import glob

In [4]:
import requests
import json
import sys

#overpass_url = "http://lz4.overpass-api.de/api/interpreter"
overpass_url = "http://z.overpass-api.de/api/interpreter"
api = overpass.API(timeout=600)

In [5]:
#COLORS
TBLUE  = '\033[34m'
TRED  = '\033[91m'
TBLACK = '\033[30m'
TWHITE = '\033[37m'
TGREEN =  '\033[92m' 
BOLD = '\033[1m'

## Queries Overpass (Bus, Train, Airplane, Fuel Station)

In [6]:
overpass_queries = ['Bus', 'Train', 'Fuel Station']

def overpass_query(mode, accuracy, lat, long):
    if mode == 'Bus':
        overpass_query_bus = """
            [out:json];
            ( 
                node["highway"="bus_stop"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
                node["amenity"="bus_station"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
                way["highway"="bus_stop"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
                way["amenity"="bus_station"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
            );
            out body;
            """
        return overpass_query_bus
    elif mode == 'Train':
        overpass_query_train = """
            [out:json];
            ( 
                node["railway"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
                way["railway"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
            );
            out body;
            """
        return overpass_query_train
    elif mode == 'Airplane':
        overpass_query_airplane = """
            [out:json];
            ( 
                node["aeroway"="aerodrome"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
                way["aeroway"="aerodrome"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
            );
            out body;
            """
        return overpass_query_airplane
    elif mode == 'Fuel_station':
        overpass_query_fuel_station = """
            [out:json];
            ( 
                node["amenity"="fuel"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
                way["amenity"="fuel"](around:"""+str(accuracy)+""","""+str(lat)+""", """+str(long)+""");
            );
            out body;
            """
        return overpass_query_fuel_station

### Requesting to Overpass

In [7]:
from requests.exceptions import ConnectionError
import time as tm

start_time = tm.time()
#-----------------------------------------------------------------

path = './users_Baleares' # use your path
all_files = glob.glob(path + "/*.csv")

for csv_user in all_files:
    user = csv_user.split('/')[-1]
    df_aux = pd.read_csv(csv_user, sep = '\t')
    df_aux['Bus'] = False
    df_aux['Train'] = False
    #df_aux['Airplane'] = False
    df_aux['Fuel Station'] = False
    
    print(TBLUE + csv_user)

    for i, row in df_aux.iterrows():
        timestamp = row['Time Stamp']
        time = row['Date Time']
        lat = row['Latitude']
        long = row['Longitude']
        
        for k in overpass_queries:
            accuracy = 50

            try:
                response = requests.get(overpass_url, params={'data': overpass_query(k, accuracy, lat, long)})
            except ConnectionError as e:
                # Whoops it wasn't a 200
                print(TRED+'Overpass Server reeboting...\nConnectionError: '+str(e)+'\n\n')
                tm.sleep(60*5)
                continue
                
            try:
                data = response.json()
            except ValueError:
                #print(row)
                #print(TBLACK+str(lat), str(long))
                #print(TBLACK+str(time))
                #print(TRED+'JSONDecodeError')
                continue

            if data['elements']:
                #print(TGREEN+""+k+" tag found!")
                df_aux.at[i, k] = True
                #print(TBLACK+str(lat), str(long))
                #print(TBLACK+str(time))
                #print(TGREEN+str(response)) #<Response [200]>
                #print(TWHITE+str(data)+'\n\n')

        df_aux.to_csv("./users_Baleares_tags/"+user, sep='\t', index=False)
        
#-----------------------------------------------------------------
total_time = tm.time() - start_time
print(str(total_time/3600) + " horas")

./users_Baleares/100549.csv
Overpass Server reeboting...
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


./users_Baleares/101468.csv
./users_Baleares/10226.csv
./users_Baleares/102671.csv
./users_Baleares/103768.csv
./users_Baleares/104290.csv
./users_Baleares/104819.csv
./users_Baleares/10560.csv
./users_Baleares/105697.csv
./users_Baleares/107459.csv
./users_Baleares/108104.csv
./users_Baleares/108635.csv
./users_Baleares/110792.csv
./users_Baleares/111518.csv
./users_Baleares/112479.csv
./users_Baleares/114512.csv
./users_Baleares/114904.csv
./users_Baleares/116479.csv
Overpass Server reeboting...
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


./users_Baleares/118617.csv
./users_Baleares/119278.csv
./users_Baleares/119282.csv
./users_Baleares/119326.csv
./users_Baleares/119509.csv
./users_Baleares/120311.csv
./users_Baleares/123886.csv
./users_Baleares/124

In [ ]:
ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))